# Discovery

## Earthquake dataset

In [1]:
import pandas as pd

df = pd.read_csv("data/earthquake.csv")
df.head()

,Date,Time,Latitude,Longitude,Type,Depth,Depth Error,Depth Seismic Stations,Magnitude,Magnitude Type,...,Magnitude Seismic Stations,Azimuthal Gap,Horizontal Distance,Horizontal Error,Root Mean Square,ID,Source,Location Source,Magnitude Source,Status
0,01/02/1965,13:44:18,19.246,145.616,Earthquake,131.6,NaN,NaN,6.0,MW,...,NaN,NaN,NaN,NaN,NaN,ISCGEM860706,ISCGEM,ISCGEM,ISCGEM,Automatic
1,01/04/1965,11:29:49,1.863,127.352,Earthquake,80.0,NaN,NaN,5.8,MW,...,NaN,NaN,NaN,NaN,NaN,ISCGEM860737,ISCGEM,ISCGEM,ISCGEM,Automatic
2,01/05/1965,18:05:58,-20.579,-173.972,Earthquake,20.0,NaN,NaN,6.2,MW,...,NaN,NaN,NaN,NaN,NaN,ISCGEM860762,ISCGEM,ISCGEM,ISCGEM,Automatic
3,01/08/1965,18:49:43,-59.076,-23.557,Earthquake,15.0,NaN,NaN,5.8,MW,...,NaN,NaN,NaN,NaN,NaN,ISCGEM860856,ISCGEM,ISCGEM,ISCGEM,Automatic
4,01/09/1965,13:32:50,11.938,126.427,Earthquake,15.0,NaN,NaN,5.8,MW,...,NaN,NaN,NaN,NaN,NaN,ISCGEM860890,ISCGEM,ISCGEM,ISCGEM,Automatic


In [2]:
df.columns

Index(['Date', 'Time', 'Latitude', 'Longitude', 'Type', 'Depth', 'Depth Error',
       'Depth Seismic Stations', 'Magnitude', 'Magnitude Type',
       'Magnitude Error', 'Magnitude Seismic Stations', 'Azimuthal Gap',
       'Horizontal Distance', 'Horizontal Error', 'Root Mean Square', 'ID',
       'Source', 'Location Source', 'Magnitude Source', 'Status'],
      dtype='object')

In [3]:
# Let's say we want to use the dataset to identify earthquake characteristics using common attribute,
# so we only need to select relevant columns
columns = [
    "Date", "Time", "Latitude", "Longitude", "Type",
    "Magnitude", "Magnitude Error", "Depth",
    "Depth Error"
]

df = df[columns]
df.head()

,Date,Time,Latitude,Longitude,Type,Magnitude,Magnitude Error,Depth,Depth Error
0,01/02/1965,13:44:18,19.246,145.616,Earthquake,6.0,NaN,131.6,NaN
1,01/04/1965,11:29:49,1.863,127.352,Earthquake,5.8,NaN,80.0,NaN
2,01/05/1965,18:05:58,-20.579,-173.972,Earthquake,6.2,NaN,20.0,NaN
3,01/08/1965,18:49:43,-59.076,-23.557,Earthquake,5.8,NaN,15.0,NaN
4,01/09/1965,13:32:50,11.938,126.427,Earthquake,5.8,NaN,15.0,NaN


## Tectonic plate dataset

In [4]:
import geopandas as gpd

gdf_plate = gpd.read_file("data/tectonic_plates.json")
gdf_plate.head()

,LAYER,Code,PlateName,geometry
0,plate,AF,Africa,"POLYGON ((-0.4379 -54.8518, -0.91466 -54.4535,..."
1,plate,AN,Antarctica,"POLYGON ((180 -65.7494, 180 -90, -180 -90, -18..."
2,plate,SO,Somalia,"POLYGON ((32.1258 -46.9998, 32.1252 -46.9975, ..."
3,plate,IN,India,"POLYGON ((56.2652 14.6232, 57.0015 14.6601, 57..."
4,plate,AU,Australia,"MULTIPOLYGON (((-180 -32.30415, -180 -15.62071..."


# Cleaning

## Handle duplicated value

In [5]:
df.duplicated().sum()

np.int64(0)

There are no duplicated rows

## Handle missing value

In [6]:
df.isna().sum()

Date                   0
Time                   0
Latitude               0
Longitude              0
Type                   0
Magnitude              0
Magnitude Error    23085
Depth                  0
Depth Error        18951
dtype: int64

In [7]:
missing_cols = [
    "Depth Error", "Magnitude Error"
]

df[missing_cols].describe()

,Depth Error,Magnitude Error
count,4461.000000,327.000000
mean,4.993115,0.071820
std,4.875184,0.051466
min,0.000000,0.000000
25%,1.800000,0.046000
50%,3.500000,0.059000
75%,6.300000,0.075500
max,91.295000,0.410000


For the missing columns only consist error metrics. There could be two scenarios:
1. `NaN` value means there is no error calculation.
2. `NaN` value truly represents a missing value.

Let's assume the second scenario and replace it with median value.

In [8]:
# For numerical cols, replace missing value with median, since it's less affected by outlier
df["Magnitude Error"] = df["Magnitude Error"].fillna(df["Magnitude Error"].median())

df["Depth Error"] = df["Depth Error"].fillna(df["Depth Error"].median())

In [9]:
df.isna().sum()

Date               0
Time               0
Latitude           0
Longitude          0
Type               0
Magnitude          0
Magnitude Error    0
Depth              0
Depth Error        0
dtype: int64

# Structuring

## Combine `Date` and `Time` columns into datetime

In [10]:
df["Datetime"] = pd.to_datetime(df["Date"] + " " + df["Time"], format="%m/%d/%Y %H:%M:%S", errors="coerce")

df.head()

,Date,Time,Latitude,Longitude,Type,Magnitude,Magnitude Error,Depth,Depth Error,Datetime
0,01/02/1965,13:44:18,19.246,145.616,Earthquake,6.0,0.059,131.6,3.5,1965-01-02 13:44:18
1,01/04/1965,11:29:49,1.863,127.352,Earthquake,5.8,0.059,80.0,3.5,1965-01-04 11:29:49
2,01/05/1965,18:05:58,-20.579,-173.972,Earthquake,6.2,0.059,20.0,3.5,1965-01-05 18:05:58
3,01/08/1965,18:49:43,-59.076,-23.557,Earthquake,5.8,0.059,15.0,3.5,1965-01-08 18:49:43
4,01/09/1965,13:32:50,11.938,126.427,Earthquake,5.8,0.059,15.0,3.5,1965-01-09 13:32:50


In [11]:
converted_coerce_datetime = pd.to_datetime(df[df["Datetime"].isna()]["Date"]).dt.floor("s").dt.tz_localize(None)
df.loc[converted_coerce_datetime.index, "Datetime"] = converted_coerce_datetime

df.head()

,Date,Time,Latitude,Longitude,Type,Magnitude,Magnitude Error,Depth,Depth Error,Datetime
0,01/02/1965,13:44:18,19.246,145.616,Earthquake,6.0,0.059,131.6,3.5,1965-01-02 13:44:18
1,01/04/1965,11:29:49,1.863,127.352,Earthquake,5.8,0.059,80.0,3.5,1965-01-04 11:29:49
2,01/05/1965,18:05:58,-20.579,-173.972,Earthquake,6.2,0.059,20.0,3.5,1965-01-05 18:05:58
3,01/08/1965,18:49:43,-59.076,-23.557,Earthquake,5.8,0.059,15.0,3.5,1965-01-08 18:49:43
4,01/09/1965,13:32:50,11.938,126.427,Earthquake,5.8,0.059,15.0,3.5,1965-01-09 13:32:50


Since we already get the `Date` and `Time` in `Datetime`, so we can remove those two columns to make it less redundant.

In [12]:
df = df.drop(columns=["Date", "Time"])

df.head()

,Latitude,Longitude,Type,Magnitude,Magnitude Error,Depth,Depth Error,Datetime
0,19.246,145.616,Earthquake,6.0,0.059,131.6,3.5,1965-01-02 13:44:18
1,1.863,127.352,Earthquake,5.8,0.059,80.0,3.5,1965-01-04 11:29:49
2,-20.579,-173.972,Earthquake,6.2,0.059,20.0,3.5,1965-01-05 18:05:58
3,-59.076,-23.557,Earthquake,5.8,0.059,15.0,3.5,1965-01-08 18:49:43
4,11.938,126.427,Earthquake,5.8,0.059,15.0,3.5,1965-01-09 13:32:50


## Rename columns

In [13]:
df = df.rename(columns={
    "Latitude": "latitude",
    "Longitude": "longitude",
    "Type": "type",
    "Magnitude": "magnitude",
    "Magnitude Type": "magnitude_type",
    "Magnitude Error": "magnitude_error",
    "Depth": "depth",
    "Depth Error": "depth_error",
    "Datetime": "datetime"
})

df.head()

,latitude,longitude,type,magnitude,magnitude_error,depth,depth_error,datetime
0,19.246,145.616,Earthquake,6.0,0.059,131.6,3.5,1965-01-02 13:44:18
1,1.863,127.352,Earthquake,5.8,0.059,80.0,3.5,1965-01-04 11:29:49
2,-20.579,-173.972,Earthquake,6.2,0.059,20.0,3.5,1965-01-05 18:05:58
3,-59.076,-23.557,Earthquake,5.8,0.059,15.0,3.5,1965-01-08 18:49:43
4,11.938,126.427,Earthquake,5.8,0.059,15.0,3.5,1965-01-09 13:32:50


# Enriching

## Add `energy_released` column

In [14]:
# formula log E = 5.24 + 1.44Mw -> E = 10^(5.24 + 1.44Mw)

df["energy_released"] = df["magnitude"].apply(lambda x: 10**(5.24 + 1.44*x))

df.head()

,latitude,longitude,type,magnitude,magnitude_error,depth,depth_error,datetime,energy_released
0,19.246,145.616,Earthquake,6.0,0.059,131.6,3.5,1965-01-02 13:44:18,7.585776e+13
1,1.863,127.352,Earthquake,5.8,0.059,80.0,3.5,1965-01-04 11:29:49,3.908409e+13
2,-20.579,-173.972,Earthquake,6.2,0.059,20.0,3.5,1965-01-05 18:05:58,1.472313e+14
3,-59.076,-23.557,Earthquake,5.8,0.059,15.0,3.5,1965-01-08 18:49:43,3.908409e+13
4,11.938,126.427,Earthquake,5.8,0.059,15.0,3.5,1965-01-09 13:32:50,3.908409e+13


## Add `tectonic_plate` column

In [15]:
gdf = gpd.GeoDataFrame(
    df, 
    geometry=gpd.points_from_xy(df.longitude, df.latitude),
    crs="EPSG:4326"  # WGS84
)
gdf.head()

,latitude,longitude,type,magnitude,magnitude_error,depth,depth_error,datetime,energy_released,geometry
0,19.246,145.616,Earthquake,6.0,0.059,131.6,3.5,1965-01-02 13:44:18,7.585776e+13,POINT (145.616 19.246)
1,1.863,127.352,Earthquake,5.8,0.059,80.0,3.5,1965-01-04 11:29:49,3.908409e+13,POINT (127.352 1.863)
2,-20.579,-173.972,Earthquake,6.2,0.059,20.0,3.5,1965-01-05 18:05:58,1.472313e+14,POINT (-173.972 -20.579)
3,-59.076,-23.557,Earthquake,5.8,0.059,15.0,3.5,1965-01-08 18:49:43,3.908409e+13,POINT (-23.557 -59.076)
4,11.938,126.427,Earthquake,5.8,0.059,15.0,3.5,1965-01-09 13:32:50,3.908409e+13,POINT (126.427 11.938)


In [16]:
gdf_result = gpd.sjoin(gdf, gdf_plate, how="left", predicate="within")

gdf_result

,latitude,longitude,type,magnitude,magnitude_error,depth,depth_error,datetime,energy_released,geometry,index_right,LAYER,Code,PlateName
0,19.2460,145.6160,Earthquake,6.0,0.059,131.60,3.5,1965-01-02 13:44:18,7.585776e+13,POINT (145.616 19.246),33,plate,MA,Mariana
1,1.8630,127.3520,Earthquake,5.8,0.059,80.00,3.5,1965-01-04 11:29:49,3.908409e+13,POINT (127.352 1.863),42,plate,BH,Birds Head
2,-20.5790,-173.9720,Earthquake,6.2,0.059,20.00,3.5,1965-01-05 18:05:58,1.472313e+14,POINT (-173.972 -20.579),15,plate,TO,Tonga
3,-59.0760,-23.5570,Earthquake,5.8,0.059,15.00,3.5,1965-01-08 18:49:43,3.908409e+13,POINT (-23.557 -59.076),7,plate,SA,South America
4,11.9380,126.4270,Earthquake,5.8,0.059,15.00,3.5,1965-01-09 13:32:50,3.908409e+13,POINT (126.427 11.938),30,plate,PS,Philippine Sea
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23407,38.3917,-118.8941,Earthquake,5.6,0.320,12.30,1.2,2016-12-28 08:22:12,2.013724e+13,POINT (-118.8941 38.3917),6,plate,NA,North America
23408,38.3777,-118.8957,Earthquake,5.5,0.260,8.80,2.0,2016-12-28 09:13:47,1.445440e+13,POINT (-118.8957 38.3777),6,plate,NA,North America
23409,36.9179,140.4262,Earthquake,5.9,0.059,10.00,1.8,2016-12-28 12:38:51,5.445027e+13,POINT (140.4262 36.9179),25,plate,OK,Okhotsk
23410,-9.0283,118.6639,Earthquake,6.3,0.059,79.00,1.8,2016-12-29 22:30:19,2.051162e+14,POINT (118.6639 -9.0283),11,plate,SU,Sunda


In [17]:
# Cleaning up join result
gdf_result = gdf_result.drop(columns=["geometry", "index_right", "LAYER"])

gdf_result = gdf_result.rename(columns={
    "Code": "plate_code",
    "PlateName": "plate_name"
})

gdf_result.head()

,latitude,longitude,type,magnitude,magnitude_error,depth,depth_error,datetime,energy_released,plate_code,plate_name
0,19.246,145.616,Earthquake,6.0,0.059,131.6,3.5,1965-01-02 13:44:18,7.585776e+13,MA,Mariana
1,1.863,127.352,Earthquake,5.8,0.059,80.0,3.5,1965-01-04 11:29:49,3.908409e+13,BH,Birds Head
2,-20.579,-173.972,Earthquake,6.2,0.059,20.0,3.5,1965-01-05 18:05:58,1.472313e+14,TO,Tonga
3,-59.076,-23.557,Earthquake,5.8,0.059,15.0,3.5,1965-01-08 18:49:43,3.908409e+13,SA,South America
4,11.938,126.427,Earthquake,5.8,0.059,15.0,3.5,1965-01-09 13:32:50,3.908409e+13,PS,Philippine Sea


In [18]:
# Since some of the events is not only caused by earthquake, e.g. explosion
# therefore we need to exclude the tectonic plate in that events
non_earthquake_mask = gdf_result["type"] != "Earthquake"
non_earthquake_events = gdf_result[non_earthquake_mask]

non_earthquake_events.head()

,latitude,longitude,type,magnitude,magnitude_error,depth,depth_error,datetime,energy_released,plate_code,plate_name
565,37.302167,-116.408333,Nuclear Explosion,5.62,0.245,1.2,31.61,1966-12-20 15:30:01,2.151791e+13,NA,North America
897,37.295333,-116.455667,Nuclear Explosion,5.63,0.125,1.2,31.61,1968-04-26 15:00:02,2.224334e+13,NA,North America
1129,37.231500,-116.473667,Nuclear Explosion,5.52,0.219,1.4,31.61,1968-12-19 16:30:01,1.544543e+13,NA,North America
1380,37.314167,-116.460667,Nuclear Explosion,5.82,0.187,1.2,31.61,1969-09-16 14:30:01,4.176380e+13,NA,North America
1532,37.300500,-116.534167,Nuclear Explosion,5.54,0.410,1.2,31.61,1970-03-26 19:00:01,1.650441e+13,NA,North America


In [19]:
non_earthquake_events.loc[: , ["plate_code", "plate_name"]] = None

In [20]:
gdf_result[non_earthquake_mask] = non_earthquake_events

gdf_result[non_earthquake_mask].head()

,latitude,longitude,type,magnitude,magnitude_error,depth,depth_error,datetime,energy_released,plate_code,plate_name
565,37.302167,-116.408333,Nuclear Explosion,5.62,0.245,1.2,31.61,1966-12-20 15:30:01,2.151791e+13,None,None
897,37.295333,-116.455667,Nuclear Explosion,5.63,0.125,1.2,31.61,1968-04-26 15:00:02,2.224334e+13,None,None
1129,37.231500,-116.473667,Nuclear Explosion,5.52,0.219,1.4,31.61,1968-12-19 16:30:01,1.544543e+13,None,None
1380,37.314167,-116.460667,Nuclear Explosion,5.82,0.187,1.2,31.61,1969-09-16 14:30:01,4.176380e+13,None,None
1532,37.300500,-116.534167,Nuclear Explosion,5.54,0.410,1.2,31.61,1970-03-26 19:00:01,1.650441e+13,None,None


# Validating

In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23412 entries, 0 to 23411
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   latitude         23412 non-null  float64       
 1   longitude        23412 non-null  float64       
 2   type             23412 non-null  object        
 3   magnitude        23412 non-null  float64       
 4   magnitude_error  23412 non-null  float64       
 5   depth            23412 non-null  float64       
 6   depth_error      23412 non-null  float64       
 7   datetime         23412 non-null  datetime64[ns]
 8   energy_released  23412 non-null  float64       
dtypes: datetime64[ns](1), float64(7), object(1)
memory usage: 1.6+ MB


All data types are correct

In [22]:
df.duplicated().sum()

np.int64(0)

There are no duplicated rows

In [23]:
# Final dataframe
gdf_result

,latitude,longitude,type,magnitude,magnitude_error,depth,depth_error,datetime,energy_released,plate_code,plate_name
0,19.2460,145.6160,Earthquake,6.0,0.059,131.60,3.5,1965-01-02 13:44:18,7.585776e+13,MA,Mariana
1,1.8630,127.3520,Earthquake,5.8,0.059,80.00,3.5,1965-01-04 11:29:49,3.908409e+13,BH,Birds Head
2,-20.5790,-173.9720,Earthquake,6.2,0.059,20.00,3.5,1965-01-05 18:05:58,1.472313e+14,TO,Tonga
3,-59.0760,-23.5570,Earthquake,5.8,0.059,15.00,3.5,1965-01-08 18:49:43,3.908409e+13,SA,South America
4,11.9380,126.4270,Earthquake,5.8,0.059,15.00,3.5,1965-01-09 13:32:50,3.908409e+13,PS,Philippine Sea
...,...,...,...,...,...,...,...,...,...,...,...
23407,38.3917,-118.8941,Earthquake,5.6,0.320,12.30,1.2,2016-12-28 08:22:12,2.013724e+13,NA,North America
23408,38.3777,-118.8957,Earthquake,5.5,0.260,8.80,2.0,2016-12-28 09:13:47,1.445440e+13,NA,North America
23409,36.9179,140.4262,Earthquake,5.9,0.059,10.00,1.8,2016-12-28 12:38:51,5.445027e+13,OK,Okhotsk
23410,-9.0283,118.6639,Earthquake,6.3,0.059,79.00,1.8,2016-12-29 22:30:19,2.051162e+14,SU,Sunda
